# 3.7 softmax回归的简洁实现

我们在3.3节“[线性回归的简洁实现](3.03_linear-regression-pytorch.ipynb)”中已经了解了使用Pytorch实现模型的便利。下面，让我们再次使用Pytorch来实现一个softmax回归模型。首先导入所需的包或模块。

In [1]:
import torch
from torch import nn
from torch.nn import init
import numpy as np
import sys
sys.path.append("..") 
import d2lzh_pytorch as d2l

print(torch.__version__)

1.2.0


## 3.7.1 获取和读取数据

我们仍然使用Fashion-MNIST数据集和上一节中设置的批量大小。

In [2]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

## 3.7.2 定义和初始化模型

在3.4节（[softmax回归](3.04_softmax-regression.ipynb)）中提到，softmax回归的输出层是一个全连接层，所以我们用一个线性模块就可以了。因为前面我们数据返回的每个batch样本`x`的形状为(batch_size, 1, 28, 28), 所以我们要先用`view()`将`x`的形状转换成(batch_size, 784)才送入全连接层。

In [3]:
num_inputs = 784
num_outputs = 10

class FlattenLayer(nn.Module):
    def __init__(self):
        super(FlattenLayer, self).__init__()
    def forward(self, x): # x shape: (batch, *, *, ...)
        return x.view(x.shape[0], -1)

# 方法一：直接把输入层和输出层插入到模型神经网络里，以数字为索引
net =  nn.Sequential(
        FlattenLayer(),
        nn.Linear(num_inputs, num_outputs))

print(net)

# 方法二：使用OrderedDict，建立自己的索引
# from collections import OrderedDict
# net = nn.Sequential(
#        OrderedDict([
#          ('flatten', FlattenLayer()),
#          ('linear', nn.Linear(num_inputs, num_outputs))]))

Sequential(
  (0): FlattenLayer()
  (1): Linear(in_features=784, out_features=10, bias=True)
)


In [4]:
#方法一：
init.normal_(net[1].weight, mean=0, std=0.01)
init.constant_(net[1].bias, val=0) 

#方法二：
# init.normal_(net.linear.weight, mean=0, std=0.01)
# init.constant_(net.linear.bias, val=0) 

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

## 3.7.3 softmax和交叉熵损失函数

如果做了上一节的练习，那么你可能意识到了分开定义softmax运算和交叉熵损失函数可能会造成数值不稳定。因此，PyTorch提供了一个包括softmax运算和交叉熵损失计算的函数。它的数值稳定性更好。

In [5]:
loss = nn.CrossEntropyLoss()

## 3.7.4 定义优化算法

我们使用学习率为0.1的小批量随机梯度下降作为优化算法。

In [6]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)

## 3.7.5 训练模型

接下来，我们使用上一节中定义的训练函数来训练模型。

In [ ]:
num_epochs = 5
from time import time
start = time()
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, None, None, optimizer)
print(time() - start)

training on  cuda
epoch 1, loss 0.0019, train acc 0.840, test acc 0.829


## 小结

* PyTorch提供的函数往往具有更好的数值稳定性。
* 可以使用PyTorch更简洁地实现softmax回归。

## 练习

* 尝试调一调超参数，如批量大小、迭代周期和学习率，看看结果会怎样。